# Speech to Text Testing Notebook

Dependencies

Pip (env)
- google-cloud-speech[libcst]
- ffmpeg-python
- python-dotenv


Conda 
- nb_conda_kernels

Conda (env)
- Python 3.10
- ipykernel


## Imports

In [1]:
from google.cloud import speech
import ffmpeg
import sys
import os

## Video Conversion

Using MP4 (lossy codec) can reduce the accuracy of the audio. In order to generate the best results, we will transcode the video into lossless codec (FLAC). I decided not to go with LINEAR16 since its use case lies in applications that need more higher quality audio (also results in higher file size)

## Optimal Formats
- Sampling Rate > 16k Hz\n
- Lossless Codec (FLAC)\n
- Short queries -> use `StreamingRecognize` with `single_utterance` set to `true`

In [44]:
base_dir = os.getcwd()
print(base_dir)

c:\Users\ricky\Documents\My Files\Altum Labs\autolab\sst_transcription


In [45]:
# from ipywidgets import interact
# from matplotlib import pyplot as plt
# import ipywidgets as widgets
# import numpy as np

In [21]:
# probe = ffmpeg.probe(base_dir+'/in.mp4')
# audio_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'audio']
# audio_streams #verifying that there is audio

In [46]:
!ffmpeg -i ../data/wetlab1/sec4.mp4 2>&1

ffmpeg version 2023-07-06-git-f00222e81f-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-liblensfun --en

In [62]:
# testing

input_dir = '../data/wetlab1/sec4.mp4'
output_dir = '../data/wetlab1_flac/sec4.flac'

# FLIPS VIDEO
# {
#     ffmpeg
#     .input('sec4.mp4')
#     .hflip()
#     .output('output1.mp4', acodec='copy') #acodec is supposed to copy the audio over but doesn't quite work --izzy
#     .run()
# }

#converting mp4 to audio flac file
{
    ffmpeg
    .input(input_dir)
    .output(output_dir, acodec='flac')
    .run()
}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2416979513.py, line 17)

In [48]:
!ffmpeg -i ../data/wetlab1_flac/sec4.flac 2>&1

ffmpeg version 2023-07-06-git-f00222e81f-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-liblensfun --en

## Speech2Text 

In [42]:
from googlestt import SpeechToText

In [49]:
# Get the current working directory
current_dir = os.getcwd()

# Move back one directory
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Print the parent directory
print(parent_dir)

c:\Users\ricky\Documents\My Files\Altum Labs\autolab


In [50]:
# Example of how to call STT on encoded audio bytes
stt = SpeechToText(project_id="autolab-391921",
                    recognizer_id="recognizer1")

with open(f"{parent_dir}/data/wetlab1_flac/sec4.flac", "rb") as fd:
    contents = fd.read()

    # This response contains all the output data from the model
    response = stt.speech_to_text(contents)

In [64]:
# Check docs for better description, but these two functions will
# put the results in a more convenient data type.
# First one just returns a string, prolly use this for gpt.
transcript_concat = stt.concatenate_transcripts(response)
transcript_time = stt.get_transcript_list_and_times(response)

print(transcript_concat)
print(transcript_time)

another thing I want to mention about cleaning is that if you do use any surgical lube and a cannulas if you leave these overnight the canyon as well Gunk up with the surgical Lube so it's always a good idea to take these off rinse them out or soak them in water to keep them clean otherwise you'll have to have new ones and if you have a tip for the syringe you can actually just cap that and then you have it ready to use for next time and then of course
[('another thing I want to mention about cleaning is that if you do use any surgical lube and a cannulas if you leave these', 0.0, 7.0), (" overnight the canyon as well Gunk up with the surgical Lube so it's always a good idea to take these off rinse them out or soak them in water to keep them clean otherwise you'll have to have new ones and if you have a tip for the syringe you can actually just cap that and then you have it ready to use for next time", 7.0, 24.0), (' and then of course', 24.0, 26.0)]


In [67]:
# convert transcript_time into string
format_transcript_time = ""
for item in transcript_time:
    text = item[0]
    start_time = item[1]
    end_time = item[2]
    format_transcript_time += f"{text} [{start_time}-{end_time}]\n"
print(format_transcript_time)

another thing I want to mention about cleaning is that if you do use any surgical lube and a cannulas if you leave these [0.0-7.0]
 overnight the canyon as well Gunk up with the surgical Lube so it's always a good idea to take these off rinse them out or soak them in water to keep them clean otherwise you'll have to have new ones and if you have a tip for the syringe you can actually just cap that and then you have it ready to use for next time [7.0-24.0]
 and then of course [24.0-26.0]



In [70]:
if os.path.exists(parent_dir + "/instruction_generator/data/transcript_concat.txt"):
    print("File 'transcript_concat.txt' already exists. Aborting.")
else:
    # place them in txt files for future use
    with open(parent_dir + "/instruction_generator/data/transcript_concat.txt", "w") as file:
        file.write(transcript_concat)
        print('Done!')

if os.path.exists(parent_dir + "/instruction_generator/data/transcript_time.txt"):
    print("File 'transcript_time.txt' already exists. Aborting.")
else:
    # place them in txt files for future use
    with open(parent_dir + "/instruction_generator/data/transcript_time.txt", "w") as file:
        file.write(format_transcript_time)
        print('Done!')

Done!
Done!


In [53]:
print(parent_dir + "/instruction/data/transcript_concat.txt")

c:\Users\ricky\Documents\My Files\Altum Labs\autolab/transcript_generator/data/transcript_concat.txt


In [61]:
print(f'Saved to \"%s\"' % (parent_dir + "/instruction/data/transcript_concat.txt"))

Saved to "c:\Users\ricky\Documents\My Files\Altum Labs\autolab/transcript_generator/data/transcript_concat.txt"!


In [63]:
# transcript_time

formatted_string = ""
for item in transcript_time:
    text = item[0]
    start_time = item[1]
    end_time = item[2]
    formatted_string += f"{text} [{start_time}-{end_time}]\n"

print(formatted_string)

another thing I want to mention about cleaning is that if you do use any surgical lube and a cannulas if you leave these [0.0-7.0]
 overnight the canyon as well Gunk up with the surgical Lube so it's always a good idea to take these off rinse them out or soak them in water to keep them clean otherwise you'll have to have new ones and if you have a tip for the syringe you can actually just cap that and then you have it ready to use for next time [7.0-24.0]
 and then of course [24.0-26.0]



In [ ]:
The following transcript of a lab experiement has text with start and end times of when they were said in a video. Edit the transcript into a clean and concise lab procedure that would appear in a lab report that contains the start and end times in each of the bullet points: 